In [ ]:
!pip install essentia mido torchaudio demucs

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-e4y2wk3h
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-e4y2wk3h
  Resolved https://github.com/m-bain/whisperx.git to commit 2d9ce44329ae73af2520196d31cd14b6192ace44
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Henrique e Juliano - Nada, Nada - DVD Novas Histórias - Ao vivo em Recife.mp3 to Henrique e Juliano - Nada, Nada - DVD Novas Histórias - Ao vivo em Recife.mp3


In [ ]:
import os
file_name = list(uploaded.keys())[0]  # Pega o nome do arquivo enviado
base_name = os.path.splitext(file_name)[0]

NameError: name 'uploaded' is not defined

In [ ]:
# 3. Separar voz e instrumental com Demucs
from demucs import pretrained
from demucs.apply import apply_model
import torchaudio

In [ ]:
model = pretrained.get_model('htdemucs') #esse é o modelo pré treinad

In [ ]:
wav, sr = torchaudio.load(file_name) #carregando o audio

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [ ]:
# Se o áudio tiver mais de 2 canais, reduzir para estéreo
if wav.shape[0] > 2:
    wav = wav[:2]

In [ ]:
sources = apply_model(model, wav[None], device="cuda")[0] #separando fontes

In [ ]:
torchaudio.save(f"{base_name}Voz.wav", sources[3], sr)          # Voz
torchaudio.save(f"{base_name}Instrumental.wav", sources[0]+sources[1]+sources[2], sr)  # Resto
print("Separação concluída! Arquivos gerados: voz.wav e instrumental.wav")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:247: UserWarning: torio.io._streaming_media_encoder.StreamingMediaEncoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s 

Separação concluída! Arquivos gerados: voz.wav e instrumental.wav


#FLUXO COM ESSENTIA
Passo 1 → WhisperX no vocals.wav → JSON com palavras e tempos.
Passo 2 → Essentia no mesmo vocals.wav → vetor de pitch (Hz) com timestamp. Passo 3 → Para cada palavra, pega o pitch médio no intervalo (ex.: “amor” entre 12.0–12.8s).
Passo 4 → Converte para nota MIDI → gera os eventos (Lyric_t, Note_on, Note_off).
Passo 5 → Exporta em MIDI → pronto para player karaokê.

In [ ]:
import librosa
import whisperx

# 1️⃣ Modelo WhisperX
model_name = "small.en"
device = "cuda"
model = whisperx.load_model(model_name, device=device)

# 2️⃣ Carregar áudio de voz
audio_file = f"{base_name}Voz.wav"
y, sr = librosa.load(audio_file, sr=16000)

# 3️⃣ Dividir áudio em chunks de 30s
chunk_len = 30  # segundos
chunks = []
num_chunks = int(len(y) / (chunk_len*sr)) + 1
for i in range(num_chunks):
    start = i * chunk_len * sr
    end = min((i+1) * chunk_len * sr, len(y))
    chunks.append(y[start:end])

# 4️⃣ Transcrever cada chunk separadamente
all_segments = []
for i, chunk in enumerate(chunks):
    # salvar temporariamente o chunk como WAV
    temp_file = f"chunk_{i}.wav"
    import soundfile as sf
    sf.write(temp_file, chunk, sr)

    segments, info = model.transcribe(temp_file)
    all_segments.extend(segments)

# 5️⃣ Mostrar resultados
for segment in all_segments:
    print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")

/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed fr

>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.12/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cu126. Bad things might happen unless you revert torch to 1.x.


NameError: name 'base_name' is not defined

In [ ]:
import whisper

model = whisper.load_model("small")
result = model.transcribe("Voz.wav")
for segment in result["segments"]:
    print(segment["start"], segment["end"], segment["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import essentia.standard as es

# Detectar tom da música (ex: voz separada pelo Demucs ou instrumental)
key = es.KeyExtractor()(f"{base_name}.wav")
print("Tonalidade da música:", key[0], key[1])  # Ex: ('C', 'major')

# Detectar pitch da voz do usuário
loader = es.MonoLoader(filename="UsuarioVoz.wav")
audio = loader()
pitch_extractor = es.PitchYin(frameSize=2048, hopSize=256)
pitches = []
confidences = []
for frame in es.FrameGenerator(audio, frameSize=2048, hopSize=256):
    pitch, confidence = pitch_extractor(frame)
    pitches.append(pitch)
    confidences.append(confidence)

print("Primeiros pitches detectados (Hz):", pitches[:20])


NameError: name 'base_name' is not defined